In [3]:
%matplotlib inline
from __future__ import print_function
import matplotlib.pyplot as plt
from matplotlib import cm
import numpy as np

SAMPLE_IMAGE = './RGB.byte.jpg'

# Module 4: Raster processing in Python

## GIS in Python

There has been a lot of growth in Python packages recently associated with spatial analysis
* Traditional GIS vendors like esri, QGIS have created APIs to expose functionality:
  * esri (`arcpy`)
  * QGIS (`pyqgis`)
* New geospatial companies are porting workflows to Python:
  * Mapbox (`rasterio`)
  * Stamen Design (`tilestache`)
  * DigitalGlobe 
* Google Earth Engine has Python API for massive cloud-based raster processing
  * `earthengine-api`

## Vector packages
While we won't focus on vector-based analysis in this module, there are a number of great packages for vector-based processing
* `ogr` within `osgeo`
* `fiona`
* `shapely`
* `geopandas`
* `arcpy`

## Raster packages in this module: `arcpy` and `rasterio`

We'll be looking at two different Python raster packages for this module:
* `arcpy`: Python API for ArcGIS (closed-source)
* `rasterio`: A 'pythonic' port of GDAL functionality (open-source)

My advice for usage:
* If you want a fully-functional raster analysis library with low barrier to entry, go with `arcpy`
* If you want full control over raster cell processing (e.g. custom modeling), go with `rasterio`

## Foundational elements: GDAL and `numpy`
* GDAL (C/C++): Translator library for raster and vector geospatial data formats with some utility functions
  * used for I/O of "serialized" data into memory
* numpy (Python): Fundamental package in the scientific stack of Python tools for array manipulation.
  * used as the in-memory representation of raster data
  
Because this workshop is focused on "user-facing" packages, GDAL and numpy don't get enough credit for being the engines that drive this functionality.

## arcpy
* Very large library of GIS functionality (vector, raster, cartography, data management, etc.)
* Most of `arcpy` *raster* functionality within Spatial Analyst toolbox and module (`arcpy.sa`)
* Need to "check out" Spatial Analyst extension before using
* Typically need to set up a workspace before processing
* Rasters need to be saved out using the `save` method

In [2]:
# Typical set up for arcpy environment
import arcpy
from arcpy import sa
from arcpy import env

# Check out extension
arcpy.CheckOutExtension('Spatial')

# Set up workspace
env.workspace = '.'

# Set up scratch workspace
env.scratchWorkspace = 'd:/matt'

# Allow overwriting - BE CAREFUL!!!!
env.overwriteOutput = True

## arcpy - sample functionality 

Using the `Con` method - equivalent to `if DEM > 1000, set equal to 1, else set to 0`

In [147]:
out = sa.Con('./dem.tif', 1, 0, 'value > 1000')
out.save('./dem_con.tif')

Calculating slope percent of a raster

In [148]:
out = sa.Slope('./dem.tif')
out.save('./slope_percent.tif')

Combining two rasters together - each combination is a new value in the output raster

In [151]:
out = sa.Combine(['./dem_con.tif', './slope_percent.tif'])
out.save('./combine.tif')

Dozens more functions exist - see http://desktop.arcgis.com/en/arcmap/10.5/tools/spatial-analyst-toolbox

## arcpy - in-chain processing

Instead of saving out a number of intermediate rasters, functions can be chained together to create output.  This is the same output as before, but only writing out the final raster.

In [154]:
con_raster = sa.Con('./dem.tif', 1, 0, 'value > 1000')
slope_raster = sa.Slope('./dem.tif')
combine_raster = sa.Combine([con_raster, slope_raster])
combine_raster.save('./combine_2.tif')

Or even more succinctly:

In [155]:
sa.Combine([
    sa.Con('./dem.tif', 1, 0, 'value > 1000'),
    sa.Slope('./dem.tif')
]).save('./combine_3.tif')

## arcpy - complex raster algebra

When we create a statement like `slope_raster = sa.Slope('./dem.tif')`, we are actually creating a temporary raster object.  We can use this to create complicated raster expressions with normal operators.  This following example creates three 5-class rasters based on natural breaks (DEM, aspect, and slope) and creates an algebraic expression so that each digit in the output raster represents a different class.

In [163]:
(
    sa.Slice(sa.Slope('./dem.tif'), 5, 'NATURAL_BREAKS') + 
    sa.Slice(sa.Aspect('./dem.tif'), 5, 'NATURAL_BREAKS') * 10 +
    sa.Slice('./dem.tif', 5, 'NATURAL_BREAKS') * 100
).save('combined_topography.tif')


## arcpy - working with numpy arrays
There may be times when your spatial processing task is too complicated for existing functionality within ArcGIS.  In those cases, `arcpy` allows conversion to/from rasters to `numpy` arrays using two methods:
* arcpy.RasterToNumPyArray
* arcpy.NumPyArrayToRaster

Here's an example to calculate the percentage contribution of each cell to the row's total, something not simple to do with ArcGIS functionality

In [17]:
# Convert from raster to numpy array
arr = arcpy.RasterToNumPyArray('./dem.tif')

# Calculate row totals and reshape to 2-D array
row_sums = arr.sum(axis=1).reshape((-1, 1))

# Calculate cell contributions to row totals
prop_cell = (1.0 * arr) / row_sums

# Recast as raster and save
arcpy.NumPyArrayToRaster(prop_cell).save('prop_cell.tif')

## rasterio

* fairly new package (2013)
* in contrast to `arcpy`, `rasterio` more tightly couples with both GDAL and `numpy`
* thin wrapper around the GDAL C/C++ library, mostly using its functionality for raster reading and writing
* very much a lower-level interface to raster data - no raster analysis functions built in

## rasterio - reading in data
This is very similar to `arcpy.RasterToNumPyArray` functionality.  The `pixel_values` object in the code below is a `bands x rows x cols` numpy array.

In [36]:
import rasterio
src = rasterio.open(SAMPLE_IMAGE)
pixel_values = src.read()
print(type(pixel_values))
print(pixel_values.shape)

<type 'numpy.ndarray'>
(3, 718, 791)


Reading in only one band of data - return value is a `rows x cols` numpy array

In [37]:
# Warning - bands are 1-indexed, numpy arrays are 0-indexed
pixel_values_band = src.read(1)
print(type(pixel_values_band))
print(pixel_values_band.shape)

<type 'numpy.ndarray'>
(718, 791)


## rasterio - reading in metadata

`rasterio` holds all metadata associated with a raster in the `profile` attribute.  Notice that this particular dataset has no coordinate reference system (`crs`).

In [38]:
# Print out all keys/values of the profile
for k, v in src.profile.items():
    print(k, ':', v)
    
# Dimensions of data - rows x columns
print('-' * 50)
print('Rows, columns: ', src.height, src.width)

count : 3
crs : None
interleave : pixel
dtype : uint8
photometric : ycbcr
driver : JPEG
transform : | 1.00, 0.00, 0.00|
| 0.00, 1.00, 0.00|
| 0.00, 0.00, 1.00|
height : 718
width : 791
tiled : False
nodata : None
compress : jpeg
--------------------------------------------------
Rows, columns:  718 791


## rasterio - iterating over raster "blocks"
One advantage of `rasterio` (over `arcpy`) for fine control of spatial processing and memory management is the concept of raster blocks.  Instead of reading an entire raster into memory all at once, `rasterio` provides an way to "iterate" over all the "blocks" in a raster.  Blocks are sections in a raster that are contiguous in a file system, so reading data naturally by blocks avoids unnecessary scans on a file.  It also allows for parallel processing of different raster blocks.

In [39]:
# Figure out the natural block size for an image
block_windows = src.block_windows(1)

# Iterate over these block windows and print out a
# small subarray of the returned numpy array
# The `ji` value is the row and column index of this block
# In this case, each block is a row of data
for ji, window in block_windows:
    r = src.read(1, window=window)
    print(ji, r[0, 300:304])

(0, 0) [0 0 0 0]
(1, 0) [0 0 0 0]
(2, 0) [0 0 0 0]
(3, 0) [0 0 0 0]
(4, 0) [0 0 0 0]
(5, 0) [0 0 0 0]
(6, 0) [0 0 0 0]
(7, 0) [0 0 0 0]
(8, 0) [0 0 0 0]
(9, 0) [0 0 0 0]
(10, 0) [0 0 0 0]
(11, 0) [0 0 0 0]
(12, 0) [0 0 0 0]
(13, 0) [0 0 0 0]
(14, 0) [0 0 0 0]
(15, 0) [0 0 0 0]
(16, 0) [0 0 0 0]
(17, 0) [0 0 0 0]
(18, 0) [0 0 0 0]
(19, 0) [0 0 0 0]
(20, 0) [0 0 0 0]
(21, 0) [0 0 0 0]
(22, 0) [0 0 1 1]
(23, 0) [0 0 1 1]
(24, 0) [0 2 1 1]
(25, 0) [0 0 1 1]
(26, 0) [5 0 0 0]
(27, 0) [0 0 2 0]
(28, 0) [254 250   2   1]
(29, 0) [254 254 252 255]
(30, 0) [250 254 255 255]
(31, 0) [252 250 254 232]
(32, 0) [251 241 252 251]
(33, 0) [217  68 251  80]
(34, 0) [ 23  14  15 190]
(35, 0) [ 32  16  15 117]
(36, 0) [20  8 14 14]
(37, 0) [10 12  5 10]
(38, 0) [11  9  8  8]
(39, 0) [ 7  1  3 98]
(40, 0) [ 5 10 56 83]
(41, 0) [11  3 11  8]
(42, 0) [9 4 9 6]
(43, 0) [6 5 8 8]
(44, 0) [11  9  5  4]
(45, 0) [ 6  5  5 11]
(46, 0) [6 9 5 5]
(47, 0) [ 3 10  9  9]
(48, 0) [ 9 12  8 14]
(49, 0) [119  45  50  72

(649, 0) [27 17 25 22]
(650, 0) [30 28 26 23]
(651, 0) [27 29 21 26]
(652, 0) [28 25 26 26]
(653, 0) [28 26 29 27]
(654, 0) [35 28 23 26]
(655, 0) [47 44 42 40]
(656, 0) [0 0 0 0]
(657, 0) [0 0 0 0]
(658, 0) [0 0 0 0]
(659, 0) [0 0 0 0]
(660, 0) [0 0 0 0]
(661, 0) [0 0 0 1]
(662, 0) [0 0 0 1]
(663, 0) [0 0 0 1]
(664, 0) [0 0 0 0]
(665, 0) [0 0 0 0]
(666, 0) [0 0 0 0]
(667, 0) [0 0 0 0]
(668, 0) [0 0 0 0]
(669, 0) [0 0 0 0]
(670, 0) [0 0 0 0]
(671, 0) [0 0 0 1]
(672, 0) [0 0 0 0]
(673, 0) [0 0 0 0]
(674, 0) [0 0 0 0]
(675, 0) [0 0 0 0]
(676, 0) [0 0 0 0]
(677, 0) [0 0 0 0]
(678, 0) [0 0 0 0]
(679, 0) [0 0 0 0]
(680, 0) [0 0 0 0]
(681, 0) [0 0 0 0]
(682, 0) [0 0 0 0]
(683, 0) [0 0 0 0]
(684, 0) [0 0 0 0]
(685, 0) [0 0 0 0]
(686, 0) [0 0 0 0]
(687, 0) [0 0 0 0]
(688, 0) [0 0 0 0]
(689, 0) [0 0 0 0]
(690, 0) [0 0 0 0]
(691, 0) [0 0 0 0]
(692, 0) [0 0 0 0]
(693, 0) [0 0 0 0]
(694, 0) [0 0 0 0]
(695, 0) [0 0 0 0]
(696, 0) [0 0 0 0]
(697, 0) [0 0 0 0]
(698, 0) [0 0 0 0]
(699, 0) [0 0 0 0]
(70

## Displaying images in Jupyter (interlude)
Often, when working in Jupyter we don't want to have to be switching on and off between Jupyter and (e.g.) ArcGIS to visualize our output.  We can use some simple image display functionality from `matplotlib` to visualize intermediate products easily.  

Here we are going to create two functions to display a single band raster and a three-band RGB raster.  We will be able to call them after we've defined them.  These won't make a lot of sense right now (until the graphics module), so we'll skip over them quickly.

In [40]:
# Create a function to display a single-band image using
# the specified colormap.  For information on matplotlib colormaps
# see this page: https://matplotlib.org/users/colormaps.html
def display_band(image, band=0, cmap=cm.gray):
    # Extract the band from the image
    arr = image[band]
    
    # Set the plot figure size
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(111)

    # Show the image
    im = ax.imshow(arr, cmap=cmap, interpolation='none')

    # Show a corresponding colorbar
    fig.colorbar(im);
    
# Create a function to display a three-band image in RGB
def display_rgb(image):
    assert(image.shape[-1] == 3)
    
    # Set the plot figure size
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(111)

    # Show the image
    im = ax.imshow(image)
    
# Function to move the first axis to the end
def move_axis(arr, from_axis=0, to_axis=-1):
    return np.moveaxis(arr, from_axis, to_axis)    

## rasterio - displaying images 
We can visualize our `rasterio` rasters (really `numpy` arrays at this point) using these new functions.  Because the sample raster that we are using is relatively small, we can read the whole raster into memory (called `pixel_values`).

This shows the first band of the 3-D numpy array.

In [ ]:
pixel_values = src.read()
display_band(pixel_values, band=0)

This shows the RGB of the image, but we first have to move the first axis to the last using the `move_axis` function

In [ ]:
display_rgb(move_axis(pixel_values))

Because we can slice a numpy array, we can also show subwindows of the raster.  Here we show the RGB image for rows and columns 400-600.

In [ ]:
subset = pixel_values[:, 400:601, 400:601]
display_rgb(move_axis(subset))

## rasterio - writing output

* Simple band arithmetic (NDVI as example?)
* Focal functions

## Exercises

### Exercise #1: Using `arcpy` in some context

### Exercise #2: Prediction using RF model from before

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Read in the ecoplot data
ecoplots_df = pd.read_csv('ecoplots.csv')

# Set which environmental variables to use as predictors - all but FCID
x_vars = [
    'B3', 'B4', 'B5', 'DEM', 'ASPTR', 'SLPPCT', 'TPI450', 'ANNPRE',
    'SMRTMP', 'SMRVP'
]

# Convert to 2-D numpy array
X = np.array(ecoplots_df[x_vars])

# Convert to 1-D numpy array
Y = np.array(ecoplots_df['SERIES'])

rf = RandomForestClassifier(n_estimators=500)
rf.fit(X, Y)

# Run 10-fold cross-validation with the training data
# scores = cross_val_score(rf, X, Y, cv=10)
# print('Cross-validation scores:', scores)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [2]:
from pyimpute import load_training_vector, load_targets, impute, evaluate_clf

c:\users\gregorma\envs\default\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
import os
ROOT = 'D:/matt/process/new/small2'
rasters = [os.path.join(ROOT, x.lower() + '.tif') for x in x_vars]
rasters

['D:/matt/process/new/small2\\b3.tif',
 'D:/matt/process/new/small2\\b4.tif',
 'D:/matt/process/new/small2\\b5.tif',
 'D:/matt/process/new/small2\\dem.tif',
 'D:/matt/process/new/small2\\asptr.tif',
 'D:/matt/process/new/small2\\slppct.tif',
 'D:/matt/process/new/small2\\tpi450.tif',
 'D:/matt/process/new/small2\\annpre.tif',
 'D:/matt/process/new/small2\\smrtmp.tif',
 'D:/matt/process/new/small2\\smrvp.tif']

In [4]:
target_xs, raster_info = load_targets(rasters)

c:\users\gregorma\envs\default\lib\site-packages\ipykernel_launcher.py:1: RasterioDeprecationWarning: 'src.affine' is deprecated.  Please switch to 'src.transform'. See https://github.com/mapbox/rasterio/issues/86 for details.
  """Entry point for launching an IPython kernel.
c:\users\gregorma\envs\default\lib\site-packages\ipykernel_launcher.py:1: RasterioDeprecationWarning: 'src.affine' is deprecated.  Please switch to 'src.transform'. See https://github.com/mapbox/rasterio/issues/86 for details.
  """Entry point for launching an IPython kernel.
c:\users\gregorma\envs\default\lib\site-packages\ipykernel_launcher.py:1: RasterioDeprecationWarning: 'src.affine' is deprecated.  Please switch to 'src.transform'. See https://github.com/mapbox/rasterio/issues/86 for details.
  """Entry point for launching an IPython kernel.
c:\users\gregorma\envs\default\lib\site-packages\ipykernel_launcher.py:1: RasterioDeprecationWarning: 'src.affine' is deprecated.  Please switch to 'src.transform'. See 

In [5]:
evaluate_clf(rf, X, Y)

Accuracy Score: 0.741588

Classification report
             precision    recall  f1-score   support

          6       0.69      0.41      0.51        22
          8       0.83      0.83      0.83        47
          9       0.98      1.00      0.99        50
         10       0.73      0.72      0.73        46
         11       0.77      0.82      0.79        44
         12       0.75      0.43      0.55        21
         13       0.82      0.88      0.85        48
         14       0.51      0.59      0.55        46
         15       0.78      0.83      0.81        48
         19       0.60      0.62      0.61        50
         20       0.50      0.40      0.44        50
         21       0.76      0.91      0.83        53
         22       0.61      0.81      0.69        53
         23       0.76      0.64      0.70        59
         25       0.81      0.68      0.74        50
         30       0.95      0.93      0.94        56

avg / total       0.74      0.74      0.74       

In [5]:
impute(target_xs, rf, raster_info, outdir='D:/matt/process/outputs_rf')

In [6]:
dir()

['In',
 'Out',
 'ROOT',
 'RandomForestClassifier',
 'X',
 'Y',
 '_',
 '_1',
 '_3',
 '__',
 '___',
 '__builtin__',
 '__builtins__',
 '__doc__',
 '__name__',
 '__package__',
 '_dh',
 '_i',
 '_i1',
 '_i2',
 '_i3',
 '_i4',
 '_i5',
 '_i6',
 '_ih',
 '_ii',
 '_iii',
 '_oh',
 '_sh',
 'cross_val_score',
 'ecoplots_df',
 'evaluate_clf',
 'exit',
 'get_ipython',
 'impute',
 'load_targets',
 'load_training_vector',
 'np',
 'os',
 'pd',
 'quit',
 'raster_info',
 'rasters',
 'rf',
 'target_xs',
 'x',
 'x_vars']